Airbnb scraper by APIFY
https://apify.com/dtrungtin/airbnb-scraper

## Dataset structure

Here we are dealing with database with few columns (we specified excel-friendly format). I suppose you can get more information and complex dataset by changing that option

This dataset was extracted on **08-June-2023** and the scrape period was **6 months**

`dataset_airbnb-scraper_2023-06-08_14-57-41-189.json`

|Column|Type|Description|
|------|----|-----------|
|url|str|URL of the listing|
|name|str|Name of the listing|
|stars|float|Star grading of the listing|
|numberOfGuests|int|Max number of occupants|
|address|str|City, State, Country|
|roomType|str|Type of the listing (e.g., Full apartment, house)|
|location|dict|`{lat, lng}`|
|reviews|list||
|pricing|dict|Dictionary containing currency, rate, etc|
|photos|list|URL's with photos of the listing|
|primaryHost|dict|Details of the host|
|additionalHosts|list|More Details about the host|
|isHostedBySuperhost|bool|Is the host SuperHost?|
|isAvailable|bool|Is the listing Available?|
|calendar|list|List with pairs of values `{Available, Date}`|
|occupancyPercentage|float|Percentage of occupancy in the specified period|

* Table made with the information extracted from the `preliminar.ipynb` notebook 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_json('dataset_airbnb-scraper_2023-06-08_14-57-41-189.json')
data.columns

Index(['url', 'name', 'stars', 'numberOfGuests', 'address', 'roomType',
       'location', 'reviews', 'pricing', 'photos', 'primaryHost',
       'additionalHosts', 'isHostedBySuperhost', 'isAvailable', 'calendar',
       'occupancyPercentage'],
      dtype='object')

In [7]:
df = data[['stars', 'numberOfGuests', 'roomType', 'location', 'pricing',
            'isHostedBySuperhost', 'occupancyPercentage']].copy()
df.head()

,stars,numberOfGuests,roomType,location,pricing,isHostedBySuperhost,occupancyPercentage
0,4.88,4,Alojamiento entero: apto. residencial,"{'lat': 6.204, 'lng': -75.564}","{'rate': {'amount': 159, 'amountFormatted': '$...",False,0.97
1,NaN,3,Alojamiento entero: apto. residencial,"{'lat': 6.24723, 'lng': -75.59545}","{'rate': {'amount': 237, 'amountFormatted': '$...",False,3.40
2,4.69,5,Alojamiento entero: apartamento con servicios,"{'lat': 6.201, 'lng': -75.574}","{'rate': {'amount': 188, 'amountFormatted': '$...",True,16.99
3,4.19,6,Alojamiento entero: piso,"{'lat': 6.21169, 'lng': -75.57166}","{'rate': {'amount': 180, 'amountFormatted': '$...",False,3.40
4,NaN,2,Habitación privada en: bed and breakfast,"{'lat': 6.20033, 'lng': -75.56914}","{'rate': {'amount': 176, 'amountFormatted': '$...",True,13.11


In [8]:
data.location[0]

{'lat': 6.204, 'lng': -75.564}

In [9]:
def typeofproperty(p):
    if 'hotel' in p:
        return 'hotel'
    elif 'habit' in p or 'piso' in p or 'loft' in p or 'privada' in p or 'apartamento' in p or 'apto. residencial' in p:
        return 'apartamento'
    elif 'casa' in p or 'granja' in p or 'villa' in p or 'adosado' in p:
        return 'casa'
    elif 'entero' in p and 'vivienda' in p:
        return 'casa'



df['pricepernight'] = df['pricing'].apply(lambda x: x['rate']['amount'])
df['propertyType'] = df.roomType.apply(typeofproperty)

df['superhost']  = df.isHostedBySuperhost.apply(lambda x: 1 if x == True else 0)
df['latitude'] = df.location.apply(lambda x: x['lat'])
df['longitude'] = df.location.apply(lambda x: x['lng'])
df.drop(columns=['pricing', 'roomType', 'location', 'isHostedBySuperhost'], inplace=True)
df

,stars,numberOfGuests,occupancyPercentage,pricepernight,propertyType,superhost,latitude,longitude
0,4.88,4,0.97,159,apartamento,0,6.20400,-75.56400
1,NaN,3,3.40,237,apartamento,0,6.24723,-75.59545
2,4.69,5,16.99,188,apartamento,1,6.20100,-75.57400
3,4.19,6,3.40,180,apartamento,0,6.21169,-75.57166
4,NaN,2,13.11,176,apartamento,1,6.20033,-75.56914
...,...,...,...,...,...,...,...,...
1234,4.96,5,21.36,54,apartamento,1,6.15719,-75.60836
1235,4.78,4,6.80,54,apartamento,1,6.19552,-75.57851
1236,4.50,6,22.33,57,apartamento,0,6.20688,-75.56564
1237,4.82,2,0.97,50,apartamento,1,6.20928,-75.55877


In [54]:
df.describe()
    

,stars,numberOfGuests,occupancyPercentage,pricepernight,superhost,latitude,longitude
count,1115.000000,1239.000000,1239.000000,1239.000000,1239.000000,1239.000000,1239.000000
mean,4.780170,4.168684,25.568475,126.456013,0.493947,6.207146,-75.570225
std,0.197935,2.516143,25.119167,93.726613,0.500165,0.025904,0.015205
min,3.500000,1.000000,0.000000,50.000000,0.000000,5.873290,-75.698340
25%,4.700000,2.000000,4.850000,64.000000,0.000000,6.200000,-75.573685
50%,4.820000,4.000000,17.960000,92.000000,0.000000,6.208180,-75.568000
75%,4.920000,5.000000,39.320000,144.500000,1.000000,6.212060,-75.564175
max,5.000000,16.000000,100.000000,500.000000,1.000000,6.268900,-75.480790


In [6]:
df.corr()

/var/folders/w3/99h1kkws5yj6c5ys1nc854gc0000gn/T/ipykernel_2813/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,stars,numberOfGuests,occupancyPercentage,pricepernight,superhost,latitude,longitude
stars,1.000000,-0.032801,0.069640,0.050697,0.414078,0.003538,0.044176
numberOfGuests,-0.032801,1.000000,0.045062,0.593515,-0.156761,-0.276945,0.001434
occupancyPercentage,0.069640,0.045062,1.000000,-0.130454,0.047826,-0.017641,-0.011657
pricepernight,0.050697,0.593515,-0.130454,1.000000,-0.117911,-0.189060,0.127666
superhost,0.414078,-0.156761,0.047826,-0.117911,1.000000,0.157378,-0.034236
latitude,0.003538,-0.276945,-0.017641,-0.189060,0.157378,1.000000,0.025626
longitude,0.044176,0.001434,-0.011657,0.127666,-0.034236,0.025626,1.000000


In [53]:
import folium
import branca

# Create a map centered at an initial point
map = folium.Map(location=[6.225, -75.5812], zoom_start=14)

# Create a colormap for the color coding
colormap = branca.colormap.linear.YlOrRd_09.scale(df.pricepernight.min(), df.pricepernight.max())


for i, row in df.iterrows():
    #folium.Marker([row['latitude'], row['longitude']]).add_to(map)
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']], radius=5,
        color=colormap(row['pricepernight']),
        fill=True,
        fill_color=colormap(row['pricepernight']),
        fill_opacity=0.7
    ).add_to(map)

map
